In [1]:
import sys
import os

import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy import stats

sys.path.insert(0, "../../scs")
import data_preparation as dp
import data_plotting as dplt
import data_degrading as dg

from icecream import ic
from importlib import reload

rng = np.random.RandomState(1415)

In [2]:
def preproccess_dataframe(sn_data, phase_range, ptp_range, wvl_range):
    # The function below neatly and reproducibly extracts all of the relevant 
    # subsets of the dataframe.
    data = dp.extract_dataframe(sn_data)
    wvl0 = data[1]  # Wavelength array
    flux0_columns = data[2]  # Columns that index the fluxes in the dataframe
    fluxes0 = data[6]  # Only the flux values in a numpy array

    # Spectra with a spectral phase outside of `phase_range`.
    bad_ind = sn_data["Spectral Phase"] < phase_range[0]
    bad_ind |= sn_data["Spectral Phase"] > phase_range[1]

    # Remove the spectra with a peak to valley that is too small or too large.
    ptp = np.ptp(fluxes0, axis=1)
    bad_ind |= ptp < ptp_range[0]
    bad_ind |= ptp > ptp_range[1]
    
    # Remove spectra that are completely flat within `wvl_range` (all the same value) for some reason.
    wvl_range_mask = (wvl0 < wvl_range[0]) | (wvl0 > wvl_range[1])
    flat_spectra_inds = np.std(fluxes0, axis=1, where=~wvl_range_mask) == 0
    bad_ind |= flat_spectra_inds

    # `bad_ind` now is True for all rows that we want to remove, so now we set `fluxes0` to only the rows we want.
    fluxes0 = fluxes0[~bad_ind]
    
    standardized_fluxes0 = standardize_fluxes(fluxes0, wvl0, wvl_range)

    # Set the standardized flux data into the dataframe using `~bad_ind` to index only the rows that were not removed.
    sn_data.loc[~bad_ind, flux0_columns] = standardized_fluxes0

    # Remove the rows that we have pruned above.
    sn_data = sn_data.loc[~bad_ind]

    return sn_data

def standardize_fluxes(fluxes, wvl, wvl_range):
    # Use `wvl_range` to define a mask that is True outside of `wvl_range` and False inside.
    wvl_range_mask = (wvl < wvl_range[0]) | (wvl > wvl_range[1])

    # Standardize the dataset to zero mean and standard deviation of 1.
    flux_means = np.mean(fluxes, axis=1, where=~wvl_range_mask)[..., None]
    flux_stds = np.std(fluxes, axis=1, where=~wvl_range_mask)[..., None]
    standardized_fluxes = (fluxes - flux_means) / flux_stds

    # Set all flux values outside of `wvl_range` to 0.
    standardized_fluxes[:, wvl_range_mask] = 0
    
    # Check that the standardization worked.
    assert np.all(standardized_fluxes[:, wvl_range_mask] == 0), "All data points outside of `wvl_range` should be 0."
    
    mean_inside_wvl_range = np.mean(standardized_fluxes, axis=1, where=(~wvl_range_mask))
    assert np.all(np.isclose(mean_inside_wvl_range, 0)), "Mean of all data points inside `wvl_range` should be 0."
    
    stdv_inside_wvl_range = np.std(standardized_fluxes, axis=1, where=(~wvl_range_mask))
    assert np.all(np.isclose(stdv_inside_wvl_range, 1)), "Stddev of all data points inside `wvl_range` should be 1."
    
    return standardized_fluxes


def clasificar(clase):
    if clase == 'Ia-norm':
        return 0
    elif clase.startswith('Ia'):
        return 1
    else:
        return 2


def df_split(x, train_frac, rng):
    x["Exclude"] = False
    x["Training Set"] = False

    sn_names = x.index.unique().to_list()
    num_supernova = len(sn_names)
    if num_supernova == 1:
        x["Exclude"] = True
        return x

    num_train = int(np.ceil(num_supernova * train_frac))
    if num_supernova - num_train == 0:
        num_train -= 1

    inds = rng.choice(sn_names,
                      size=num_train,
                      replace=False)
    x.loc[inds, "Training Set"] = True
    return x


def split_data(sn_data, train_frac, rng):

    sn_data_split = sn_data.groupby(by=["SN Subtype"],
                                    axis=0,
                                    group_keys=True).apply(df_split,
                                                           train_frac,
                                                           rng)
    training_set = sn_data_split["Training Set"] & ~sn_data_split["Exclude"]
    testing_set = ~sn_data_split["Training Set"] & ~sn_data_split["Exclude"]
    sn_data_trn = sn_data_split.loc[training_set]
    sn_data_tst = sn_data_split.loc[testing_set]

    sn_data_trn.reset_index(level="SN Subtype", drop=True, inplace=True)
    sn_data_tst.reset_index(level="SN Subtype", drop=True, inplace=True)
    return sn_data_trn, sn_data_tst

def gen_redshift(spectrum, rng):
    """Loosely simulate redshift estimation error by shifting the entire spectrum by at most 5 pixels left or right."""
    shift_amount = stats.randint.rvs(-5, 6, size=1, random_state=rng).item()
    shifted_spectrum = np.roll(spectrum, shift_amount)
    return shifted_spectrum


def augment(sn_data, wvl_range):
    
    # Unpack the dataframe. The dataframe is quite dense, information-wise, so this function unpacks the various pieces of information in a consistent way.
    data = dp.extract_dataframe(sn_data)
    wvl0 = data[1]  # The wavelength array of the spectra
    fluxes = data[6]  # The fluxes as a numpy array.

    # Generate a mask for the spectrum which is False within the wavelength range specified by `wvl_range`, and True otherwise. This way, we can use this mask after the data augmentation steps to make sure that all of the fluxes outside of `wvl_range` are 0.
    wvl_range_mask = (wvl0 < wvl_range[0]) | (wvl0 > wvl_range[1])
    
    # Figure out how many times each spectrum should be repeated when augmenting the dataset.
    sn_types, num_spectra = np.unique(sn_data["New Classification"], return_counts=True)
    num_augments = (np.max(num_spectra) - num_spectra) / num_spectra
    num_augments = np.ceil(num_augments).astype(int) + 1
    print(num_augments)
    
    ic(sn_types)
    ic(num_augments)
    
    # Loop through each supernova type, performing the augmentation steps.
    sn_type_df_list = []
    for sn_type, num_augment in zip(sn_types, num_augments):
        # Grab the subset of the dataframe which includes only the rows corresponding to `sn_type`. Call it `sn_type_df`.
        # We first make a copy of the original dataframe so that there is no chance of overwriting the original data with pointers/views.
        df_copy = sn_data.copy(deep=True)
        sn_type_mask = sn_data["New Classification"] == sn_type
        sn_type_df = df_copy[sn_type_mask]

        # This generates a dataframe which repeats all of the data in `sn_type_df` the number of times specified by `num_augment`. This repeated data forms the basis for the dataset augmentation.
        sn_type_df_rep = sn_type_df.iloc[np.tile(np.arange(sn_type_df.shape[0]), num_augment)].copy(deep=True)

        # Unpack the dataframe to grab the fluxes and the `flux_columns` which will be used to index the dataframe later on. Note that `flux_columns` is the exact same as the wavelength array but are strings.
        data = dp.extract_dataframe(sn_type_df_rep)
        flux_columns = data[2]
        fluxes = data[6]
        
        # Generate redshifted dataset
        shifted = gen_redshift(fluxes, rng)
        
        # Generate noise to be added to the data.
        #noise = gen_noise(fluxes, rng)
        
         # Generate spikes to be added to the data.
       #spikes = gen_spikes(fluxes, rng)
        
        # Construct the augmented dataset by replacing fluxes with the shifted fluxes, adding noise and spikes.
        augmented_fluxes = shifted 
        
        # Reset the fluxes outside of `wvl_range` to 0.
        augmented_fluxes[:, wvl_range_mask] = 0
        
        # Put these augmented fluxes back into `sn_type_df_rep` which up until this point still only contained the original fluxes without augmentation.
        sn_type_df_rep.loc[:, flux_columns] = augmented_fluxes
        
        # Appending `sn_type_df_rep` to a list allows us to later use `pd.concat` to combine all of them together after the for loop is complete, creating the final augmented dataset in one simple line of code.
        sn_type_df_list.append(sn_type_df_rep)
        
    # Finally, we have the augmented dataset.
    sn_data_augmented = pd.concat(sn_type_df_list, axis=0)
        
    # Lastly we must re-standardize the data such that each spectrum has mean 0 and standard deviation 1.
    data = dp.extract_dataframe(sn_data_augmented)
    fluxes_aug_columns = data[2]
    fluxes_aug = data[6]
    
    standardized_fluxes_aug = standardize_fluxes(fluxes_aug, wvl0, wvl_range)

    # Set the data in the dataframe to be the re-standardized data.
    sn_data_augmented.loc[:, fluxes_aug_columns] = standardized_fluxes_aug

    return sn_data_augmented

In [3]:
#Primero tengo que leer el parquet y re-etiquetar la columna de SN
#Luego tengo que buscar el redshift de cada una, pasarlo a Amstrong y definir un rango de 0 a x pixeles para que corra
#Luego de eso separar el set de entrenamiento y el de testeo
#Luego generar el aumento de datos y ya 
#Luego de eso generar todas las imagenes de entrenamiento con el redshift corrido

In [4]:
data = pd.read_parquet('sn_data.parquet')
data

,SN Subtype,SN Subtype ID,SN Maintype,SN Maintype ID,Spectral Phase,2501.69,2505.08,2508.48,2511.87,2515.28,...,9872.21,9885.59,9898.98,9912.39,9925.82,9939.27,9952.73,9966.21,9979.71,9993.24
SN Name,,,,,,,,,,,,,,,,,,,,,
sn2008ar,Ia-norm,0,Ia,0,-8.50,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sn2008ar,Ia-norm,0,Ia,0,-7.50,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sn2008ar,Ia-norm,0,Ia,0,-6.60,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sn2008ar,Ia-norm,0,Ia,0,-4.60,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sn2008ar,Ia-norm,0,Ia,0,-3.70,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sn2007uy,Ib-pec,9,Ib,1,12.82,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sn2007uy,Ib-pec,9,Ib,1,45.82,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sn2007uy,Ib-pec,9,Ib,1,54.82,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
data['New Classification'] = data['SN Subtype'].apply(clasificar)


In [6]:
data['New Classification'].value_counts()

0    2387
2    1713
1     901
Name: New Classification, dtype: int64

In [7]:
#Separación set de entranamiento y testeo 

train_frac = 0.30
df_P_trn, df_P_tst = split_data(data, train_frac, rng)

In [8]:
df_P_trn['New Classification'].value_counts()

0    730
2    542
1    305
Name: New Classification, dtype: int64

In [9]:
#Generar aumento de datos con los datos ->

In [10]:
wvl_range = (4500, 7000)
df_PA_trn = augment(df_P_trn, wvl_range)
df_PA_trn

ic| sn_types: array([0, 1, 2])
ic| num_augments: array([1, 3, 2])


[1 3 2]


,SN Subtype,SN Subtype ID,SN Maintype,SN Maintype ID,Spectral Phase,2501.69,2505.08,2508.48,2511.87,2515.28,...,9912.39,9925.82,9939.27,9952.73,9966.21,9979.71,9993.24,New Classification,Exclude,Training Set
SN Name,,,,,,,,,,,,,,,,,,,,,
sn06kf,Ia-norm,0,Ia,0,-7.655,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,False,True
sn06kf,Ia-norm,0,Ia,0,-1.747,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,False,True
sn06kf,Ia-norm,0,Ia,0,18.673,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,False,True
sn2004bk,Ia-norm,0,Ia,0,6.400,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,False,True
sn2001ep,Ia-norm,0,Ia,0,-7.000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
sn2005ek,Ic-pec,12,Ic,2,0.100,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,False,True
sn2005ek,Ic-pec,12,Ic,2,1.100,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,False,True
sn2005ek,Ic-pec,12,Ic,2,2.100,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,False,True


In [11]:
df_PA_trn['New Classification'].value_counts()

2    1084
1     915
0     730
Name: New Classification, dtype: int64